<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Model_Backtest_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninsta

In [20]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

# 1. Configuration
SMA_RANGE = range(3, 21)
BB_PERIODS = [10, 20, 30]
SYMBOLS = ["TSLA", "SPY", "QQQ", "NVDA", "META", "MSTR", "COIN", "GLD", "AMD", "SLV", "PLTR", "MSFT", "ORCL", "IWM", "AAPL", "AVGO", "AMZN", "UNH", "NFLX", "MU", "GOOGL", "TSM", "LULU", "CRWV", "GOOG", "IBIT", "JPM", "HOOD", "GDX", "ADBE", "NOW", "APP", "GS", "WOLF", "BABA", "IREN", "COST", "INTC", "LLY", "CRCL", "CVNA", "SNDK", "OKLO", "SMH", "BA", "BMNR", "ASTS", "NBIS", "SOFI", "BE"]

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="5y", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Generate Raw Trades with Technical Audit Columns
all_trades_list = []
print("Generating Technical Audit Logs...")

for sma_val in SMA_RANGE:
    for bb_val in BB_PERIODS:
        for symbol, df_orig in data_cache.items():
            df = df_orig.copy()

            # Indicators
            df['SMA_Value'] = ta.sma(df['Close'], length=sma_val)
            bb = ta.bbands(df['Close'], length=bb_val, std=2)
            if bb is None: continue

            bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]
            df['BB_Midpoint'] = bb[bbm_col]

            # Logic: Cross above SMA AND Below BB Midpoint
            df['Prev_Close'] = df['Close'].shift(1)
            df['Prev_SMA'] = df['SMA_Value'].shift(1)

            cond_cross = (df['Close'] > df['SMA_Value']) & (df['Prev_Close'] <= df['Prev_SMA'])
            cond_value = df['Close'] < df['BB_Midpoint']

            # Filter and Label
            trades = df[cond_cross & cond_value].copy()
            if not trades.empty:
                # Forward Returns
                trades['Ret_3D'] = df['Close'].pct_change(3).shift(-3)
                trades['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
                trades['Ret_10D'] = df['Close'].pct_change(10).shift(-10)

                # Metadata
                trades['Symbol'] = symbol
                trades['SMA_Param'] = sma_val
                trades['BB_Param'] = bb_val

                # Selecting Audit Columns
                audit_cols = [
                    'Symbol', 'SMA_Param', 'BB_Param',
                    'Close', 'SMA_Value', 'BB_Midpoint',
                    'Ret_3D', 'Ret_5D', 'Ret_10D'
                ]
                all_trades_list.append(trades[audit_cols])

master_trade_log = pd.concat(all_trades_list)
master_trade_log = master_trade_log.reset_index() # Keep the Date column
print(f"Done! {len(master_trade_log)} total trade instances recorded.")

# VIEW SPECIFIC SYMBOL DETAIL (Example: NVDA)
display(master_trade_log[master_trade_log['Symbol'] == 'NVDA'].head(20))

Fetching fresh data...
Generating Technical Audit Logs...
Done! 87094 total trade instances recorded.


Price,Date,Symbol,SMA_Param,BB_Param,Close,SMA_Value,BB_Midpoint,Ret_3D,Ret_5D,Ret_10D
255,2021-02-24,NVDA,3,10,14.458340,14.292057,14.754981,-0.045331,-0.116853,-0.139764
256,2021-03-01,NVDA,3,10,13.802933,13.583052,14.344884,-0.106309,-0.162443,-0.046666
257,2021-03-09,NVDA,3,10,12.489450,12.158914,13.015729,0.026817,0.061580,0.043969
258,2021-03-26,NVDA,3,10,12.807663,12.641324,12.934751,0.039644,0.089433,0.184571
259,2021-04-23,NVDA,3,10,15.227697,15.121376,15.384485,0.000753,-0.016754,-0.029675
260,2021-05-06,NVDA,3,10,14.487273,14.408715,14.953671,-0.014925,-0.059062,0.006163
261,2021-05-14,NVDA,3,10,14.207961,13.854748,14.286816,-0.012445,0.052570,0.140525
262,2021-07-19,NVDA,3,10,18.737841,18.594081,19.709346,0.043358,0.027383,0.051665
263,2021-08-12,NVDA,3,10,19.860580,19.802378,19.972931,-0.022457,-0.005375,0.108666
264,2021-08-19,NVDA,3,10,19.753820,19.388636,19.819174,0.100768,0.114658,0.131425


In [30]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# Define display columns for consistency
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# --- SECTION A: Best Parameter Set PER Symbol ---
# Sort by Symbol and Win Rate, then take the top 1 for each ticker
best_per_symbol = performance_summary.sort_values(
    ['Symbol', 'Win_Rate_10D', 'Avg_10D_Ret'],
    ascending=[True, False, False]
).groupby('Symbol').head(1)

print("--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---")
display(best_per_symbol[display_cols].sort_values('Win_Rate_10D', ascending=False).head(25).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))


--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
308,ASTS,19,20,3,100.0%,10.23%,100.0%,12.94%,100.0%,12.54%
188,AMZN,14,10,8,37.5%,0.95%,87.5%,2.13%,100.0%,4.26%
764,CRWV,18,20,1,100.0%,2.82%,100.0%,12.24%,100.0%,20.67%
807,CVNA,16,10,6,100.0%,25.98%,83.3%,23.02%,100.0%,27.56%
387,BA,11,10,1,100.0%,0.24%,100.0%,9.43%,100.0%,8.31%
560,BMNR,17,10,1,100.0%,6.16%,100.0%,18.79%,100.0%,6.82%
702,CRCL,13,10,1,100.0%,7.75%,100.0%,4.23%,100.0%,21.12%
2370,SPY,19,20,3,66.7%,1.43%,100.0%,3.04%,100.0%,3.67%
2293,SOFI,11,10,2,50.0%,0.95%,50.0%,13.11%,100.0%,13.19%
2268,SNDK,19,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%


In [29]:
# 1. Global Aggregation: Using the Raw Trade Log
# This avoids 'average of averages' for both Win Rates AND Returns.
global_strategy_raw = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Symbol': 'count',      # Total Trade Count
    'Ret_3D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_5D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_10D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ]
})

# 2. Flatten Multi-index columns for cleaner access
global_strategy_raw.columns = [
    'Total_Trades',
    'Wins_3D', 'Avg_3D_Ret',
    'Wins_5D', 'Avg_5D_Ret',
    'Wins_10D', 'Avg_10D_Ret'
]
global_strategy_raw = global_strategy_raw.reset_index()

# 3. Recalculate Global Win Rates from raw counts
global_strategy_raw['Win_Rate_3D'] = global_strategy_raw['Wins_3D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_5D'] = global_strategy_raw['Wins_5D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_10D'] = global_strategy_raw['Wins_10D'] / global_strategy_raw['Total_Trades']

# 4. Final Formatting & Sorting
final_cols = [
    'SMA_Param', 'BB_Param', 'Total_Trades',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

global_strategy_raw = global_strategy_raw[final_cols].sort_values(by='Win_Rate_10D', ascending=False)

print("--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---")
display(global_strategy_raw.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- [B] GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---


,SMA_Param,BB_Param,Total_Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
26,12,10,306,50.3%,0.42%,55.6%,0.96%,59.2%,1.75%
32,14,10,395,52.9%,0.86%,54.4%,1.00%,57.7%,2.17%
25,11,30,1843,55.2%,0.46%,54.0%,0.47%,56.4%,1.47%
23,11,10,199,52.3%,0.93%,51.8%,0.77%,56.3%,0.82%
28,12,30,1720,55.2%,0.48%,54.7%,0.56%,56.2%,1.35%
29,13,10,370,50.8%,0.45%,53.0%,0.78%,56.2%,1.49%
24,11,20,1666,53.7%,0.38%,53.0%,0.34%,55.8%,1.26%
27,12,20,1497,53.6%,0.49%,53.8%,0.51%,55.7%,1.13%
31,13,30,1620,54.6%,0.44%,54.3%,0.47%,55.4%,1.27%
22,10,30,1947,54.9%,0.38%,53.9%,0.43%,55.4%,1.37%


In [22]:
# --- Symbol Drill-Down Section ---
target_symbol = "NVDA"  # Change this to any symbol from your list

print(f"--- Top 10 Parameter Combinations for {target_symbol} (Sorted by Avg_10D_Ret) ---")

# Filter the performance_summary we created above
symbol_drill_down = performance_summary[performance_summary['Symbol'] == target_symbol]

# Display the results
display(symbol_drill_down[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Parameter Combinations for NVDA (Sorted by Avg_10D_Ret) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
1881,NVDA,11,30,28,67.9%,1.10%,71.4%,1.87%,64.3%,5.76%
1878,NVDA,10,30,31,64.5%,1.02%,67.7%,1.72%,64.5%,5.56%
1884,NVDA,12,30,28,64.3%,0.83%,60.7%,1.31%,67.9%,5.03%
1876,NVDA,9,30,37,64.9%,1.22%,62.2%,1.70%,59.5%,5.01%
1890,NVDA,14,30,28,71.4%,2.06%,57.1%,1.47%,60.7%,4.90%
1887,NVDA,13,30,25,72.0%,1.90%,64.0%,2.11%,64.0%,4.84%
1880,NVDA,11,20,28,60.7%,0.73%,67.9%,1.51%,64.3%,4.69%
1891,NVDA,15,10,9,66.7%,1.43%,44.4%,1.31%,55.6%,4.65%
1883,NVDA,12,20,28,50.0%,0.01%,57.1%,1.27%,75.0%,4.63%
1873,NVDA,8,30,39,66.7%,1.54%,61.5%,1.43%,61.5%,4.47%


In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Prepare the Aggregate Data (if not already done in Cell 2)
param_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': 'mean',
    'Ret_5D': 'mean',
    'Ret_10D': 'mean'
}).reset_index()

# 2. Define the horizons we want to visualize
horizons = [('3D', 'Ret_3D'), ('5D', 'Ret_5D'), ('10D', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Average Returns) ---")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_table = param_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    # Create the visualization using Pandas Styler for a clean Colab output
    print(f"\n{label} Horizon Average Return:")
    display(pivot_table.style.background_gradient(cmap='RdYlGn', axis=None)
            .format("{:.2%}")
            .set_caption(f"Average {label} Return by SMA and BB Period"))

# Optional: If you prefer a more "visual" graphical heatmap using Seaborn:
# plt.figure(figsize=(10, 6))
# sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn")
# plt.title("10D Return Surface")
# plt.show()

--- PARAMETER SURFACE HEATMAPS (Average Returns) ---

3D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.47%,0.42%,0.32%
4,0.37%,0.28%,0.20%
5,-0.09%,0.19%,0.12%
6,-0.24%,0.20%,0.22%
7,-0.25%,0.11%,0.18%
8,-0.16%,0.19%,0.29%
9,-0.16%,0.18%,0.29%
10,nan%,0.31%,0.38%
11,0.93%,0.38%,0.46%



5D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.85%,0.37%,0.28%
4,0.88%,0.22%,0.19%
5,-0.15%,0.10%,0.14%
6,-0.39%,0.07%,0.20%
7,-0.28%,0.08%,0.22%
8,-0.33%,0.08%,0.29%
9,-0.16%,0.19%,0.32%
10,nan%,0.33%,0.43%
11,0.77%,0.34%,0.47%



10D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,1.06%,1.03%,1.04%
4,0.93%,0.82%,0.85%
5,0.44%,0.67%,0.77%
6,0.34%,0.87%,0.99%
7,0.64%,1.02%,1.14%
8,0.92%,1.13%,1.29%
9,0.78%,1.13%,1.32%
10,nan%,1.13%,1.37%
11,0.82%,1.26%,1.47%


In [24]:
# 1. Prepare the Win Rate Aggregate Data
# We calculate the mean of the boolean (Return > 0) to get the percentage of wins
win_rate_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': lambda x: (x > 0).mean(),
    'Ret_5D': lambda x: (x > 0).mean(),
    'Ret_10D': lambda x: (x > 0).mean()
}).reset_index()

# 2. Define the horizons for visualization
horizons = [('3-Day', 'Ret_3D'), ('5-Day', 'Ret_5D'), ('10-Day', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---")
print("Focus: Probability of a positive return across the portfolio.")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_win_rate = win_rate_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    print(f"\n{label} Win Rate Surface:")
    # Using a 0.5 (50%) midpoint for the gradient helps highlight 'Edge' vs 'Coin Flip'
    display(pivot_win_rate.style.background_gradient(cmap='RdYlGn', vmin=0.45, vmax=0.65, axis=None)
            .format("{:.1%}")
            .set_caption(f"{label} Win Rate by SMA and BB Period"))

--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---
Focus: Probability of a positive return across the portfolio.

3-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.9%,53.0%,52.3%
4,50.0%,52.5%,52.0%
5,48.8%,51.9%,51.6%
6,48.0%,52.1%,52.4%
7,47.8%,52.2%,52.6%
8,47.9%,52.7%,53.6%
9,48.2%,52.5%,53.8%
10,nan%,53.5%,54.9%
11,52.3%,53.7%,55.2%



5-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.1%,52.1%,51.8%
4,50.0%,51.5%,51.5%
5,48.5%,50.5%,51.0%
6,48.4%,51.4%,51.8%
7,49.1%,51.8%,52.2%
8,49.2%,52.4%,52.9%
9,50.0%,52.4%,53.0%
10,nan%,53.3%,53.9%
11,51.8%,53.0%,54.0%



10-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,52.8%,53.5%,53.3%
4,51.8%,52.6%,52.8%
5,50.7%,51.9%,52.2%
6,50.5%,52.5%,52.9%
7,51.1%,53.3%,53.6%
8,52.1%,53.8%,54.2%
9,52.6%,54.5%,55.2%
10,nan%,54.6%,55.4%
11,56.3%,55.8%,56.4%


In [34]:
# 1. Create a copy for export to keep the original data intact
export_df = global_strategy_raw.copy()

# 2. Round all numerical columns to 4 decimal places
# This ensures that Win Rates (0.6543 -> 0.65) and Returns (0.0123 -> 0.01) are consistent
export_df = export_df.round(4)

# 3. Rename columns for the final report
export_df = export_df.rename(columns={
    'SMA_Param': 'SMA',
    'BB_Param': 'BB',
    'Total_Trades': 'Trades'
})

# 4. Export to CSV string
# float_format="%.4f" is a safety measure to ensure trailing zeros are kept (e.g., 0.50)
csv_output = export_df.to_csv(index=False, float_format="%.4f")

print("--- COPY AND PASTE THIS TO GEMINI ---")
print(csv_output)

--- COPY AND PASTE THIS TO GEMINI ---
SMA,BB,Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
12,10,306,0.5033,0.0042,0.5556,0.0096,0.5915,0.0175
14,10,395,0.5291,0.0086,0.5443,0.0100,0.5772,0.0217
11,30,1843,0.5524,0.0046,0.5399,0.0047,0.5643,0.0147
11,10,199,0.5226,0.0093,0.5176,0.0077,0.5628,0.0082
12,30,1720,0.5523,0.0048,0.5471,0.0056,0.5622,0.0135
13,10,370,0.5081,0.0045,0.5297,0.0078,0.5622,0.0149
11,20,1666,0.5366,0.0038,0.5300,0.0034,0.5582,0.0126
12,20,1497,0.5364,0.0049,0.5377,0.0051,0.5571,0.0113
13,30,1620,0.5463,0.0044,0.5426,0.0047,0.5537,0.0127
10,30,1947,0.5485,0.0038,0.5388,0.0043,0.5537,0.0137
14,20,1196,0.5293,0.0052,0.5276,0.0053,0.5535,0.0119
14,30,1547,0.5417,0.0051,0.5372,0.0054,0.5527,0.0129
9,30,2083,0.5377,0.0029,0.5305,0.0032,0.5521,0.0132
13,20,1344,0.5268,0.0042,0.5305,0.0037,0.5506,0.0105
15,10,421,0.4964,0.0038,0.5321,0.0046,0.5463,0.0194
10,20,1804,0.5349,0.0031,0.5327,0.0033,0.5460,0.0113
9,20,1973,0.5251,0.0018,0.5241,0.00

In [35]:
# 1. Aggregate Advanced Risk Metrics from Raw Data
# We focus on the 10-Day window as it showed the highest potential for spreads
stress_test = master_trade_log.groupby(['SMA_Param', 'BB_Param'])['Ret_10D'].agg([
    ('Trades', 'count'),
    ('Avg_Ret', 'mean'),
    ('Median_Ret', 'median'),
    ('Std_Dev', 'std'),
    ('Win_Rate', lambda x: (x > 0).mean())
]).reset_index()

# 2. Calculate Expectancy Score (Assuming 1:1 Risk/Reward Ratio)
# Formula: (WinRate * 1) - (LossRate * 1)
# A score of 0.10 means you expect to make $0.10 for every $1.00 risked over time.
stress_test['Expectancy_Score'] = (stress_test['Win_Rate'] * 2) - 1

# 3. Sort by Expectancy Score
stress_test = stress_test.sort_values('Expectancy_Score', ascending=False)

print("--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---")
print("Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.")

display(stress_test.head(10).style.format({
    'Avg_Ret': '{:.2%}',
    'Median_Ret': '{:.2%}',
    'Std_Dev': '{:.2%}',
    'Win_Rate': '{:.1%}',
    'Expectancy_Score': '{:.3f}'
}).background_gradient(subset=['Expectancy_Score'], cmap='Greens'))

--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---
Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.


,SMA_Param,BB_Param,Trades,Avg_Ret,Median_Ret,Std_Dev,Win_Rate,Expectancy_Score
26,12,10,305,1.75%,1.16%,10.84%,59.2%,0.183
32,14,10,392,2.17%,0.88%,10.89%,57.7%,0.154
25,11,30,1836,1.47%,1.00%,11.60%,56.4%,0.129
23,11,10,198,0.82%,0.97%,9.84%,56.3%,0.126
28,12,30,1713,1.35%,1.00%,11.12%,56.2%,0.124
29,13,10,368,1.49%,0.80%,10.97%,56.2%,0.124
24,11,20,1660,1.26%,0.84%,11.54%,55.8%,0.116
27,12,20,1492,1.13%,0.87%,11.12%,55.7%,0.114
31,13,30,1614,1.27%,0.95%,11.34%,55.4%,0.107
22,10,30,1937,1.37%,0.92%,11.39%,55.4%,0.107
